# First part

## Wikipedia Scraping

In [1]:
import urllib.request

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
page = urllib.request.urlopen(url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(page, "lxml")

In [6]:
right_table=soup.find('table', class_='wikitable sortable')

In [7]:
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

## Pandas processing

In [59]:
import pandas as pd

In [9]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Extracting the rows with Borough not assigned :

In [10]:
right_df = df[df['Borough']!='Not assigned\n']
right_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if there is any Neighbourhood not assigned left :

In [11]:
right_df[right_df['Neighbourhood']=='Not assigned\n']

,Postal Code,Borough,Neighbourhood


There isn't ! Our dataframe is almost ready.

Just a removal of unnecessary "\n" at the end of each line, and there we go :

In [12]:
right_df = right_df.replace('\n','', regex=True)

In [13]:
right_df.shape

(103, 3)

# Second part

In [14]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011


In [15]:
import geocoder

In [16]:
latitude=[]
longitude=[]

for n in range(len(right_df)):
    
    postal_code = right_df.iloc[n,0]

    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
right_df['Latitude'] = latitude
right_df['Longitude'] = longitude

Our dataframe is ready :

In [17]:
right_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75188,-79.33036
3,M4A,North York,Victoria Village,43.73042,-79.31282
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


# Third part

In [24]:
!pip install folium

     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011


In [60]:
import folium
import requests
from pandas.io.json import json_normalize

Let's extract Toronto boroughs from the original dataframe, and create a map centered on Toronto :

In [40]:
toronto_df = right_df[right_df['Borough'].str.find("Toronto")>-1]
toronto_df.reset_index(inplace=True)
toronto_df.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,22,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
4,30,M4E,East Toronto,The Beaches,43.67703,-79.29542


In [28]:
Toronto = geocoder.arcgis('Toronto, Ontario')
Toronto_lat_lng = Toronto.latlng
latitude = Toronto_lat_lng[0]
longitude = Toronto_lat_lng[1]

In [41]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's explore a first neighborhood to see how our data is structured :

In [34]:
CLIENT_ID = 'ZYT1XJN30WBCSSST43YNCRHRB4YJUTYVSOBV23IXZKRHICRY'
CLIENT_SECRET = '0PKAPEQEHKHMBYGZNJY0WWE1S4FUVGLD5AHHIPMXCYQA1NCV'
VERSION = '20200812'

In [42]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude']
neighborhood_longitude = toronto_df.loc[0, 'Longitude']

neighborhood_name = toronto_df.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.655140000000074, -79.36264999999997.


In [43]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [46]:
results = requests.get(url).json()

In [47]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Here are the first venues found in Regent Park, Harbourfront :

In [64]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [65]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


Then we can do the same for every neighborhood :

In [66]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [68]:
print(toronto_venues.shape)
toronto_venues.head()

(1759, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [96]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 224 uniques categories.


Now we figure the frequency of each venue per neighborhood :

In [102]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['0_Neighbourhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.rename(columns={"0_Neighbourhood": "Neighbourhood"}, inplace=True)

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
toronto_onehot.shape

(1759, 225)

In [103]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,...,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.014925
1,"Brockton, Parkdale Village, Exhibition Place",0.011494,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.022989,0.011494,...,0.00,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022989
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.000000,0.010000,0.030000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.014085,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.00,0.014085
4,Central Bay Street,0.000000,0.000000,0.000000,0.00,0.000000,0.012821,0.012821,0.000000,0.000000,...,0.00,0.000000,0.000000,0.012821,0.012821,0.012821,0.000000,0.000000,0.00,0.000000
5,Christie,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.013333,0.013333,0.00,0.000000,0.000000,0.000000,0.013333,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.013333
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.040000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.010000
8,Davisville,0.000000,0.000000,0.037037,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [104]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And finally we have the top 10 venues for each neighborhood :

In [160]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Lounge,Italian Restaurant,Café,Cheese Shop,Cocktail Bar,Beer Bar,Bakery,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Sandwich Place,Nightclub,Gift Shop,Yoga Studio,Lounge,Japanese Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Italian Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Salon / Barbershop,Pub,Burrito Place,Café
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Gym / Fitness Center,Bar,Café,French Restaurant,Sandwich Place,Park,Bakery,Speakeasy
4,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Sandwich Place,Sushi Restaurant,Restaurant,Café,Italian Restaurant,Electronics Store,Middle Eastern Restaurant


## Clustering

In [131]:
from sklearn.cluster import KMeans
from collections import Counter

In [161]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

pd.DataFrame(Counter(kmeans.labels_), index=['Count'])

,5,1,2,4,3,0
Count,28,7,1,1,1,1


NB: 6 clusters is the best if we do not want to have almost all neighborhoods in one unique cluster

In [162]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,5,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Restaurant,Distribution Center
1,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,5,Coffee Shop,Café,Park,Sandwich Place,Music Store,Office,Burger Joint,Chinese Restaurant,Salon / Barbershop,Salad Place
2,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,5,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Furniture / Home Store,Theater,Bookstore,Ramen Restaurant
3,22,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557,5,Coffee Shop,Café,Seafood Restaurant,Clothing Store,Cocktail Bar,American Restaurant,Restaurant,Cosmetics Shop,Diner,Beer Bar
4,30,M4E,East Toronto,The Beaches,43.67703,-79.29542,0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [120]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [163]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally after we mapped our clusters, we can build a description of each neighborhood based on its top venues :

In [164]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [189]:
def top_venues(cluster):
    repart = dict(zip(toronto_venues['Venue Category'].unique().tolist(), np.zeros(len(toronto_venues['Venue Category'].unique()))))
    for venue in toronto_venues['Venue Category'].unique().tolist():
        liste_venues = getIndexes(cluster, venue)
        for k in range(len(liste_venues)):
            repart[venue] += 6-int(getIndexes(cluster, venue)[k][1][0])

    repart=pd.DataFrame.from_dict(repart, orient='index', columns=['Score'])
    top_df = repart.sort_values(by='Score', ascending=False).head()
    top_df = top_df/top_df.max()*5
    return top_df.round(1)

### Cluster 0

In [196]:
cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster0)

,Score
Health Food Store,5.0
Falafel Restaurant,5.0
Pub,4.0
Trail,3.0
Neighborhood,2.0


### Cluster 1

In [197]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster1)

,Score
Park,5.0
Farm,2.1
Falafel Restaurant,1.7
Playground,1.6
Sandwich Place,1.4


### Cluster 2

In [198]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster2)

,Score
Event Space,5.0
Park,5.0
Yoga Studio,4.0
Donut Shop,3.0
Fish Market,2.0


### Cluster 3

In [199]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster3)

,Score
Falafel Restaurant,5.0
Clothing Store,5.0
Yoga Studio,4.0
Dumpling Restaurant,3.0
Flea Market,2.0


### Cluster 4

In [200]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster4)

,Score
Event Space,5.0
Tennis Court,5.0
Gym,4.0
Yoga Studio,3.0
Fish Market,2.0


### Cluster 5

In [201]:
cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

top_venues(cluster5)

,Score
Coffee Shop,5.0
Café,2.9
Hotel,1.2
Italian Restaurant,0.9
Restaurant,0.9


Thank you for reviewing my notebook !